In [1]:
from spinesTS.preprocessing import split_series
from spinesTS.plotting import plot2d
from sklearn.metrics import r2_score

In [2]:
from spinesTS.data import BuiltInSeriesData
from spinesTS.nn import *
from spinesTS.metrics import mean_absolute_error
from spinesTS.metrics import mean_absolute_percentage_error

In [3]:
series_data = BuiltInSeriesData()

+----+----------------------+----------------------------------------------+
|    | table's name         | table's columns                              |
+----+----------------------+----------------------------------------------+
|  0 | ETTh1                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  1 | ETTh2                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  2 | ETTm1                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  3 | ETTm2                | date, HUFL, HULL, MUFL, MULL, LUFL, LULL, OT |
|  4 | Electric_Production  | date, value                                  |
|  5 | Messages_Sent        | date, ta, tb, tc                             |
|  6 | Messages_Sent_Hour   | date, hour, ta, tb, tc                       |
|  7 | Series_0             | date, timepoint, turnover_1, turnover_2      |
|  8 | Series_1             | date, timepoint, turnover_1, turnover_2      |
|  9 | Series_2             | date, timepoint, turnover_1, turnover_2      |

In [4]:
from spinesTS.preprocessing import MultiDimScaler

In [5]:
cs = series_data[0]
cs_data = cs.dataset
from sklearn.preprocessing import MinMaxScaler
mms = MultiDimScaler()
data = cs_data.iloc[:, 1:].values

x_train, x_test, y_train, y_test = split_series(data, cs_data['OT'], 128, 24, train_size=0.9)

x_train = mms.fit_transform(x_train, MinMaxScaler)
x_test = mms.transform(x_test)

In [6]:
x_train.shape, x_test.shape

(torch.Size([15542, 128, 7]), torch.Size([1727, 128, 7]))

## RecurrentWeightedDenseNet

### one res_dense_blocks

In [ ]:
wdr = GAUNet((128, 7), 24, random_seed=42, flip_features=False, learning_rate=0.01, level=5)
wdr.fit(x_train, y_train, eval_set=(x_test[:-2], y_test[:-2]), batch_size=32,
             min_delta=0, patience=100, epochs=1000, verbose=True, lr_scheduler='ReduceLROnPlateau')
y_pred_cs = wdr.predict(x_test[-2:])
print(f"r2: {r2_score(y_test[-2:].T, y_pred_cs.T)}")
print(f"mae: {mean_absolute_error(y_test[-2:], y_pred_cs)}")
print(f"mape: {mean_absolute_percentage_error(y_test[-2:], y_pred_cs)}")
a = plot2d(y_test[-2:], y_pred_cs, eval_slices='[-1]', labels=['y_test', 'y_pred'], figsize=(20, 6))

Epoch 1/1000 
486/486 - p0 - loss: 3.3097 - mae: 3.4671 - val_loss: 3.7624 - val_mae: 4.2531 - 5.20s/epoch - 0.011s/step
Epoch 2/1000 
486/486 - p0 - loss: 3.5813 - mae: 3.3911 - val_loss: 3.5164 - val_mae: 3.9982 - 4.82s/epoch - 0.010s/step


In [ ]:
y_pred_cs = wdr.predict(x_test)
print(f"r2: {r2_score(y_test.T, y_pred_cs.T)}")
print(f"mae: {mean_absolute_error(y_test, y_pred_cs)}")
print(f"mape: {mean_absolute_percentage_error(y_test, y_pred_cs)}")
a = plot2d(y_test, y_pred_cs, eval_slices='[:30]', labels=['y_test', 'y_pred'], figsize=(20, 10))

In [ ]:
# GAU WITHOUT FLIP
# r2: -2.461928251531084
# mae: 0.7824241320292159
# mape: 0.08264914036325263

# FLIP
# r2: -3.2216385445817077
# mae: 0.8676682909329737
# mape: 0.09149143596419052

# GAU multi variable, without flip
# r2: 0.2748685062570435
# mae: 0.3432978590329491
# mape: 0.035340427385704455

# r2: 0.32585591409789555
# mae: 0.3419693509737651
# mape: 0.035039248061682106

# LEVEL 5
# r2: 0.07409507520026454
# mae: 0.4331215620040898
# mape: 0.04468689204933249

# GAU multi variable, flip
# r2: -0.21693335422066395
# mae: 0.44913035631179854
# mape: 0.04723922629564383


# Time2Vec without flip
# r2: -3.110017119085237
# mae: 0.8972663482030238
# mape: 0.09488035043723038

# Time2Vec, flip
# r2: -3.793590596311873
# mae: 1.0186231931050622
# mape: 0.10717737492764423

# Time2Vec multi variable, without flip
# r2: -0.13839113655174196
# mae: 0.43564714988072756
# mape: 0.04561015992232642

# Time2Vec multi variable, flip
# r2: 0.06966361843654473
# mae: 0.3852565685908001
# mape: 0.039687981777916825